In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats

In [11]:
mp_holistic = mp.solutions.holistic     # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

DATA_PATH = os.path.join('MP_Data')                     # Path for exported data, numpy arrays
frames = 30                                             # Video frames

actions = np.array(['car', 'coffee', 'face', 'fall', 'friday', 'gold', 'goodbye', 'heart', 'hello', 'iloveyou', 'key', 'monday', 'music', 'phone', 'pretty', 'saturday', 'spring', 'summer', 'sunday', 'tea', 'thanks', 'thursday', 'tuesday', 'wednesday', 'winter'])
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
def mediapipe_detection(image, detection):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR to RGB
        image.flags.writeable = False                   # Ustawienie obrazu jako niezapisywalnego
        results = detection.process(image)              # Przeprowadzenie detekcji
        image.flags.writeable = True                    # Ustawienie obrazu jako zapisywalnego
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # RGB to BGR
    except Exception as e:
        print(f"Error during mediapipe detection: {e}")
        return None, None
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [13]:
model = load_model('CNN')

In [48]:
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)

# Ustawienie rozdzielczości kamery na niższą, aby poprawić wydajność
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        
        # Przetwarzanie obrazu
        image, results = mediapipe_detection(frame, holistic)
        
        # Wyodrębnianie punktów kluczowych
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        # Predykcja, gdy sekwencja znaku ma 30 klatek
        if len(sequence) == 30:
            input_data = np.expand_dims(sequence, axis=0)
            res = model.predict(input_data)[0]
            predictions.append(np.argmax(res))
            
            # Stabilizacja predykcji
            if np.unique(predictions[-10:])[0] == np.argmax(res) and res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
        
        # Wyświetlanie wyników
        cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        # Przerwanie programu po naciśnięciu klawisza 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 20ms/step


In [36]:
cap.release()
cv2.destroyAllWindows()